# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
geoapify_key

'6ed52d39092845549b9dba85ce4f2110'

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,guiuan,11.0333,125.7247,28.87,78,100,7.11,PH,1690918192
1,1,bredasdorp,-34.5322,20.0403,7.21,82,99,1.44,ZA,1690918193
2,2,port-aux-francais,-49.3500,70.2167,5.42,98,100,11.97,TF,1690918193
3,3,kharp,66.8014,65.8081,18.76,70,100,2.49,RU,1690918193
4,4,tolanaro,-25.0319,46.9987,21.04,73,40,6.17,MG,1690918193


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_height= 700, frame_width= 500, size = "Humidity", scale = 0.5, color = "City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.copy()
ideal_df = ideal_df.loc[(ideal_df["Max Temp"] > 8)]
ideal_df = ideal_df.loc[(ideal_df["Max Temp"] < 16)]

ideal_df = ideal_df.loc[(ideal_df["Wind Speed"] < 5)]
ideal_df = ideal_df.loc[(ideal_df["Cloudiness"] <= 1)]
# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,port elizabeth,-33.9180,25.5701,9.50,93,0,2.06,ZA,1690918196
37,37,port lincoln,-34.7333,135.8667,11.27,76,0,4.70,AU,1690918198
74,74,kodiak,57.7900,-152.4072,15.73,77,0,2.57,US,1690918211
105,105,kone,-21.0595,164.8658,15.03,82,0,2.33,NC,1690918218
245,245,chinchilla,-26.7500,150.6333,12.16,88,0,4.08,AU,1690918246
326,326,usinsk,65.9939,57.5281,15.99,77,0,3.95,RU,1690918261
375,375,kropachevo,55.0112,57.9896,15.06,90,0,1.39,RU,1690918271
501,501,black point,-34.6187,137.8972,10.47,75,0,4.36,AU,1690918302
539,539,goondiwindi,-28.5500,150.3167,11.18,90,0,4.06,AU,1690918308


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\ptesk\AppData\Local\Temp\ipykernel_11548\4207309733.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
29,port elizabeth,ZA,-33.9180,25.5701,93,
37,port lincoln,AU,-34.7333,135.8667,76,
74,kodiak,US,57.7900,-152.4072,77,
105,kone,NC,-21.0595,164.8658,82,
245,chinchilla,AU,-26.7500,150.6333,88,
326,usinsk,RU,65.9939,57.5281,77,
375,kropachevo,RU,55.0112,57.9896,90,
501,black point,AU,-34.6187,137.8972,75,
539,goondiwindi,AU,-28.5500,150.3167,90,


In [21]:
base_url = "https://api.geoapify.com/v2/places"
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}
latitude = '-33.9180'
longitude = '25.5701'

params["filter"] = f"circle:{longitude},{latitude},{radius}"

params["bias"] = f"proximity:{longitude},{latitude}"
name_address = requests.get(base_url, params=params)
name_address.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Waterford Hotel',
    'country': 'South Africa',
    'country_code': 'za',
    'state': 'Eastern Cape',
    'county': 'Nelson Mandela Bay Metropolitan Municipality',
    'city': 'Gqeberha',
    'postcode': '7162',
    'district': 'Steytler',
    'suburb': 'Nelson Mandela Bay Ward 11',
    'street': 'Durban Road',
    'lon': 25.5777512,
    'lat': -33.9320089,
    'state_code': 'EC',
    'formatted': 'Waterford Hotel, Durban Road, Nelson Mandela Bay Ward 11, Gqeberha, 7162, South Africa',
    'address_line1': 'Waterford Hotel',
    'address_line2': 'Durban Road, Nelson Mandela Bay Ward 11, Gqeberha, 7162, South Africa',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     

{'statusCode': 400,
 'error': 'Bad Request',
 'message': '"value" must contain at least one of [geometryId, lat1, lat, filter, bias]'}

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"

    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    response = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
port lincoln - nearest hotel: Boston Hotel
kodiak - nearest hotel: Shelikof Lodge
kone - nearest hotel: Pacifik Appartels
chinchilla - nearest hotel: Chinchilla Downtown Motor Inn
usinsk - nearest hotel: общежитие Лукойл-Коми
kropachevo - nearest hotel: МакДан
black point - nearest hotel: No hotel found
goondiwindi - nearest hotel: Queensland Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
29,port elizabeth,ZA,-33.9180,25.5701,93,Waterford Hotel
37,port lincoln,AU,-34.7333,135.8667,76,Boston Hotel
74,kodiak,US,57.7900,-152.4072,77,Shelikof Lodge
105,kone,NC,-21.0595,164.8658,82,Pacifik Appartels
245,chinchilla,AU,-26.7500,150.6333,88,Chinchilla Downtown Motor Inn
326,usinsk,RU,65.9939,57.5281,77,общежитие Лукойл-Коми
375,kropachevo,RU,55.0112,57.9896,90,МакДан
501,black point,AU,-34.6187,137.8972,75,No hotel found
539,goondiwindi,AU,-28.5500,150.3167,90,Queensland Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.5,
    color = "City",
    hover_cols = ["Lng", "Lat", "City", "Hotel Name", "Country"]
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)